#### Utiliser la bibliothèque Faker pour générer des données fictives et les indexe dans Elasticsearch, un moteur de recherche et d'analyse de données.

In [11]:
from faker import Faker         # un module qui permet de générer des données fictives         
from elasticsearch import Elasticsearch         # un module qui permet d'interagir avec Elasticsearch depuis Python

In [12]:
# Instancier le générateur de données Faker. Crée une instance de l'objet Faker pour générer les données fictives
fake = Faker()

# Connexion à Elasticsearch :
es = Elasticsearch([{'host': 'localhost', 'port':9200, 'scheme':'http'}])

# Nombre de documents fictifs à générer
num_documents = 100

# Boucle pour générer les documents et les indexer dans Elasticsearch. Chaque document est un dictionnaire contenant des clés. 
# Les valeurs de ces clés sont générées à l'aide des méthodes fournies par l'objet Faker.
for _ in range(num_documents):
    document = {
        "patient_lastname": fake.last_name(),
        "patient_firstname": fake.first_name(),
        "text": fake.text(),
        "date": fake.date_this_decade(),
        "patient_left": fake.boolean(),
        "emotion": fake.word(),
        "confidence": fake.pyfloat()
    }

    # Indexer le document dans Elasticsearch
    es.index(index="notes", body=document)


/tmp/ipykernel_16030/1515430086.py:24: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index="notes", body=document)


ApiError: ApiError(429, 'cluster_block_exception', 'index [notes] blocked by: [TOO_MANY_REQUESTS/12/disk usage exceeded flood-stage watermark, index has read-only-allow-delete block];')

In [13]:
es = Elasticsearch([{'host': 'localhost', 'port':9200, 'scheme':'http'}])

# Récupérer la version d'Elasticsearch
response = es.info()          

# Extraire la version à partir de la réponse
version = response['version']['number']

# Afficher la version
print("Version d'Elasticsearch :", version)

Version d'Elasticsearch : 7.17.10


/tmp/ipykernel_16030/3761513727.py:4: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.info()


In [29]:
# Rechercher tous les documents dans l'index "notes"
response = es.search(index="notes2", body={"query": {"match_all": {}}})      # envoie une requête pour récupérer des informations sur Elasticsearch. 
# La réponse contient les documents correspondants. Les documents sont extraits en accédant à response['hits']['hits'].


# Récupérer les documents à partir de la réponse
documents = response['hits']['hits']

# Parcourir les documents et afficher leurs données
for doc in documents:
    print("ID du document :", doc['_id'])     # L'ID du document est accessible via doc['_id'].
    print("Données du document :", doc['_source'])      # Les données du document sont accessibles via doc['_source'].
    print("--------------------")


/tmp/ipykernel_70008/2109587630.py:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(index="notes2", body={"query": {"match_all": {}}})      # envoie une requête pour récupérer des informations sur Elasticsearch.
/tmp/ipykernel_70008/2109587630.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.search(index="notes2", body={"query": {"match_all": {}}})      # envoie une requête pour récupérer des informations sur Elasticsearch.
